<h1>"Louis, I think this is the beginning of a beautiful friendship."</h1>

In [4]:
import numpy as np
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import listdir
from os.path import isfile, join
from PIL import Image
from sklearn.preprocessing import StandardScaler
import matplotlib.colors as mcolors

In [ ]:
#Reading data
print("Reading data...")
data_filenames = []
for root, dirs, files in os.walk('data/raw_images/'):  
    for filename in files:
        data_filenames.append(filename)

data = [np.array(Image.open('data/raw_images/' + filename)) for filename in data_filenames]
print("Number of raw images: \t", end="")
print(len(data))

print("\nReading annotated images of segmentation...")
annot_filenames = []
for root, dirs, files in os.walk('data/class_color/'):  
    for filename in files:
        annot_filenames.append(filename)
        
annot = [np.array(Image.open('data/class_color/' + filename)) for filename in annot_filenames ]
print("Number of annotated images: \t", end="")
print(len(annot))
if len(data)==len(annot):
    print("\nAll raw images are annotated.\n")

#Splitting data into train-validation-test parts with ratios 70-20-10
print("Splitting data into training data, validation data, test data")
nb_samples=len(data_filenames)
#Splitting ratios:
valid_split = 0.2
test_split = 0.1
train_split = 0.7
print("The ratios are: ")
print("\t train:\t", train_split )
print("\t validation::\t",valid_split )
print("\t test:\t",test_split)
    
#Splitting
data_train = np.array(data[0:int(nb_samples*(1-valid_split-test_split))])
annot_train = np.array(annot[0:int(nb_samples*(1-valid_split-test_split))])
data_valid = data[int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
annot_valid = annot[int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
data_test  = data[int(nb_samples*(1-test_split)):]
annot_test  = annot[int(nb_samples*(1-test_split)):]

#Separation of axes (RGB channels)
red_train = []
green_train = []
blue_train = []
for img in data_train:
    image = np.array(img.ravel(), dtype='float64')
    red_train.append(image[0::3])
    green_train.append(image[1::3])
    blue_train.append(image[2::3])


#Standardizing
scaler = StandardScaler()
    
print("\nStandardized data:\nRed:")
scaler.fit(np.reshape(red_train, (-1, 1)))
red_std = scaler.transform(red_train)
print(red_std)

print("\nGreen:")
scaler.fit(np.reshape(green_train, (-1,1)))
green_std = scaler.transform(green_train)
print(green_std)
    
print("\nBlue:")
scaler.fit(np.reshape(blue_train, (-1,1)))
blue_std = scaler.transform(blue_train)
print(blue_std)

print("\nNumber of training samples:\t", len(data_train))
print("Number of validation samples:\t", len(data_valid))
print("Number of test samples:\t", len(data_test))

Reading data...
Number of raw images: 	100

Reading annotated images of segmentation...
Number of annotated images: 	100

All raw images are annotated.

Splitting data into training data, validation data, test data
The ratios are: 
	 train:	 0.7
	 validation::	 0.2
	 test:	 0.1

Standardized data:
Red:


In [3]:
import tensorflow as tf
import cv2
import json
from keras.backend.tensorflow_backend import set_session
from keras.utils.np_utils import to_categorical
from tensorflow.python.client import device_lib
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
# Device check
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7281753732200905582
]
